# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. d. Brok  ->  J. d. Brok  |  ['J. d. Brok']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
F. Xu  ->  F. Xu  |  ['F. Xu']
P. Garcıa  ->  A. P. Garcia  |  ['P. Garcia']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
B. C. Estrada  ->  B. C. Estrada  |  ['B. C. Estrada']
T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 71 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.14833


extracting tarball to tmp_2511.14833...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.15079


extracting tarball to tmp_2511.15079... done.


...

extracting tarball to tmp_2511.15667...

 done.
  0: tmp_2511.15667/natbib.tex, 96 lines
  1: tmp_2511.15667/aassymbols.tex, 579 lines
  2: tmp_2511.15667/natnotes.tex, 332 lines
  3: tmp_2511.15667/sample631.tex, 900 lines


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2511.15667/sample631.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


B. C. Estrada  ->  B. C. Estrada  |  ['B. C. Estrada']
T. Henning  ->  T. Henning  |  ['T. Henning']


Found 72 bibliographic references in tmp_2511.15667/sample631.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.15079-b31b1b.svg)](https://arxiv.org/abs/2511.15079) | **The ALMA-QUARKS survey: Evidence of a candidate high-mass prestellar core aside a bright-rimmed cloud IRAS 18290-0924**  |
|| D. Yang, et al. -- incl., <mark>F. Xu</mark>, <mark>P. Garcıa</mark> |
|*Appeared on*| *2025-11-20*|
|*Comments*| *8 pages, 3 figures, accepted for publication in ApJ*|
|**Abstract**|            Although frequently reported in observations, the definitive confirmation of high-mass prestellar cores has remained elusive, presenting a persistent challenge in star formation studies. Using two-band observational data from the 3mm ATOMS and 1.3mm QUARKS surveys, we report a high-mass prestellar core candidate, C2, located on the side of the bright-rimmed cloud IRAS 18290-0924. The C2 core identified from the 3mm continuum data of the ATOMS survey ($\sim$2 arcsecond, $\rm\sim 10000~au$ at 5.3 kpc) has a mass ranging from 27-68 $M_{\odot}$ for temperatures 10-22K within a radius of $\sim$2800 au. The highest-resolution ($\sim$0.3 arcsecond, $\rm\sim 1500 au$) observations of this source presented to date from the QUARKS survey reveal no evidence of further fragmentation. Further analysis of a total $\sim$10 GHz band width of molecular line survey does not find star-formation activity (e.g., outflows, ionized gas) associated with the core, with a few molecular lines of cold gas detected only. Additionally, virial analysis indicates the C2 core is gravitationally bound ($\alpha_{\rm vir} \sim0.1-0.3$) and thus could be undergoing collapse toward star formation. These results strongly establish a candidate for a high-mass prestellar core, contributing to the very limited number of such sources known to date.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.15667-b31b1b.svg)](https://arxiv.org/abs/2511.15667) | **The JWST weather report from the nearest brown dwarfs III: Heterogeneous clouds and Thermochemical instabilities as possible drivers of WISE 1049AB's spectroscopic variability**  |
|| N. Oliveros-Gomez, et al. -- incl., <mark>B. C. Estrada</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-11-20*|
|*Comments*| *Accepted for publication in the Astrophysical Journal (ApJ). 25 pages, 11 figures, 3 tables*|
|**Abstract**|            We present a new analysis of the spectroscopic variability of WISE~J104915.57$-$531906.1AB (WISE~1049AB, L7.5+T0.5), observed using the NIRSpec instrument onboard the James Webb Space Telescope (GO 2965 - PI: Biller). We explored the variability of the dominant molecular bands present in their 0.6--5.3~$\mu$m spectra (H$_2$O, CH$_4$, CO), finding that the B component exhibits a higher maximum deviation than the A component in all the wavelength ranges tested. The light curves reveal wavelength-(atmospheric depth) and possibly chemistry-dependent variability. In particular, for the A component, the variability in the light curves at the wavelengths traced by the CH$_4$ and CO molecular absorption features is higher than that of H$_2$O, even when both trace similar pressure levels. We concluded that clouds alone are unlikely to explain the increased variability of CO and CH$_4$ with respect to H$_2$O, suggesting that an additional physical mechanism is needed to explain the observed variability. This mechanism is probably due to thermochemical instabilities. Finally, we provide a visual representation of the 3D atmospheric map reconstructed for both components using the molecular band contributions at different pressure levels and the fit of planetary-scale waves.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.14833-b31b1b.svg)](https://arxiv.org/abs/2511.14833) | **Localized Deviations from the CO-PAH Relation in PHANGS-JWST Galaxies: Faint PAH Emission or Elevated CO Emissivity?**  |
|| J. Kim, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>J. d. Brok</mark>, <mark>K. Kreckel</mark> |
|*Appeared on*| *2025-11-20*|
|*Comments*| *21 pages, 12 figures, 1 table, submitted to ApJL, comments welcome*|
|**Abstract**|            Polycyclic aromatic hydrocarbon (PAH) emission is widely used to trace the distribution of molecular gas in the interstellar medium (ISM), exhibiting a tight correlation with CO(2-1) emission across nearby galaxies. Using PHANGS-JWST and PHANGS-ALMA data, we identify localized regions where this correlation fails, with CO flux exceeding that predicted from 7.7$\mu$m PAH emission by more than an order of magnitude. These outlier regions are found in 20 out of 70 galaxies and are mostly located in galaxy centers and bars, without signs of massive star formation. We explore two scenarios to explain the elevated CO-to-PAH ratios, which can either be due to suppressed PAH emission or enhanced CO emissivity. We examine PAH emission in other bands (3.3$\mu$m and 11.3$\mu$m) and the dust continuum dominated bands (10$\mu$m and 21$\mu$m), finding consistently high CO-to-PAH (or CO-to-dust continuum) emission ratios, suggesting that 7.7$\mu$m PAH emission is not particularly suppressed. In some outlier regions, PAH size distributions and spectral energy distribution of the illuminating radiation differ slightly compared to nearby control regions with normal CO-to-PAH ratios, though without a consistent trend. We find that the outlier regions generally show higher CO velocity dispersions ($\Delta v_{\mathrm{CO}}$). This increase in $\Delta v_{\mathrm{CO}}$ may lower CO optical depth and raise its emissivity for a given gas mass. Our results favor a scenario where shear along the bar lanes and shocks at the tips of the bar elevate CO emissivity, leading to the breakdown of the CO-PAH correlation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.15470-b31b1b.svg)](https://arxiv.org/abs/2511.15470) | **Advancing Identification method of Gamma-Ray Bursts with Data and Feature Enhancement**  |
|| P. Zhang, et al. -- incl., <mark>Y. Wang</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-11-20*|
|*Comments*| *Under review. Dataset and model related discussions are welcome!*|
|**Abstract**|            Gamma-ray bursts (GRBs) are challenging to identify due to their transient nature, complex temporal profiles, and limited observational datasets. We address this with a one-dimensional convolutional neural network integrated with an Adaptive Frequency Feature Enhancement module and physics-informed data augmentation. Our framework generates 100,000 synthetic GRB samples, expanding training data diversity and volume while preserving physical fidelity-especially for low-significance events. The model achieves 97.46% classification accuracy, outperforming all tested variants with conventional enhancement modules, highlighting enhanced domain-specific feature capture. Feature visualization shows model focuses on deep-seated morphological features and confirms the capability of extracting physically meaningful burst characteristics. Dimensionality reduction and clustering reveal GRBs with similar morphologies or progenitor origins cluster in the feature space, linking learned features to physical properties. This perhaps offers a novel diagnostic tool for identifying kilonova- and supernova-associated GRB candidates, establishing criteria to enhance multi-messenger early-warning systems. The framework aids current time-domain surveys, generalizes to other rare transients, and advances automated detection in large-volume observational data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.15079/./spectrum.png', 'tmp_2511.15079/./I18290-0924_1.3mm.png', 'tmp_2511.15079/./I18290-0924_RGB.png']
copying  tmp_2511.15079/./spectrum.png to _build/html/
copying  tmp_2511.15079/./I18290-0924_1.3mm.png to _build/html/
copying  tmp_2511.15079/./I18290-0924_RGB.png to _build/html/
exported in  _build/html/2511.15079.md
    + _build/html/tmp_2511.15079/./spectrum.png
    + _build/html/tmp_2511.15079/./I18290-0924_1.3mm.png
    + _build/html/tmp_2511.15079/./I18290-0924_RGB.png
found figures ['tmp_2511.15667/./Spectrum_AB.png', 'tmp_2511.15667/./lcfit_paper_A.png', 'tmp_2511.15667/./Pressure_structures_A.png']
copying  tmp_2511.15667/./Spectrum_AB.png to _build/html/
copying  tmp_2511.15667/./lcfit_paper_A.png to _build/html/
copying  tmp_2511.15667/./Pressure_structures_A.png to _build/html/
exported in  _build/html/2511.15667.md
    + _build/html/tmp_2511.15667/./Spectrum_AB.png
    + _build/html/tmp_2511.15667/./lcfit_paper_A.png
    + _build/html/tmp_2511

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\arcm}{\hbox{^\prime}}$
$\newcommand{\etal}{{\rm et al.}\thinspace}$
$\newcommand{\eg}{{\it e.g. }}$
$\newcommand{\etc}{{\it etc. }}$
$\newcommand{\ie}{{\it i.e. }}$
$\newcommand{\cf}{{\it c.f. }}$
$\newcommand{◦ee}{\hbox{^\circ}}$
$\newcommand{\NHH}{\ensuremath{N_{\mathrm{H_{2}}}}}$
$\newcommand{\s}{\ensuremath{\mbox{~s}}}$
$\newcommand{\ps}{\ensuremath{\s^{-1}}}$
$\newcommand{\cm}{\ensuremath{\mbox{~cm}}}$
$\newcommand{\pcmsq}{\ensuremath{\cm^{-2}}}$
$\newcommand{\pcmcu}{\ensuremath{\cm^{-3}}}$
$\newcommand{\km}{\ensuremath{\mbox{~km}}}$
$\newcommand{\erg}{\ensuremath{\mbox{~erg}}}$
$\newcommand{\ergps}{\ensuremath{\erg \ps}}$
$\newcommand{\mJy}{\ensuremath{\mbox{~mJy}}}$
$\newcommand{\ML}{\ensuremath{\mbox{\Msol/\LBsol}}}$
$\newcommand{\Hi}{H\textsc{i}}$
$\newcommand{\Hii}{H\textsc{ii}}$
$\newcommand{\Ha}{\ensuremath{\mathrm{H\alpha}}}$
$\newcommand{\nh}{\ensuremath{\mathrm{n}_\mathrm{H}}}$
$\newcommand{\Mdot}{\ensuremath{\dot{\mathrm{M}}}}$
$\newcommand{\thco}{^{13}CO}$
$\newcommand{\twco}{^{12}CO}$
$\newcommand{\etco}{C^{18}O}$
$\newcommand{\vel}{km s^{-1}}$
$\newcommand{\filAname}{G350.5-N}$
$\newcommand{\filBname}{G350.5-S}$
$\newcommand{\imcoor}{\alpha_{2000}=17^{\mathrm{h}}18^{\mathrm{m}}13\fs84, \delta_{2000}=-36◦28\arcmin21\farcs5}$
$\newcommand{\her}{Herschel}$
$\newcommand{\mline}{M_{\rm line}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\um}{\mum}$
$\newcommand{\cmcm}{cm^{-2}}$
$\newcommand{\egcite}{\citep[e.g.,][]}$
$\newcommand{\lmsun}{M_{\odot}~pc^{-1}}$
$\newcommand{\chiiioh}{CH_3OH}$
$\newcommand{\hciiin}{HC_3N}$
$\newcommand{\hcop}{HCO^{+}}$
$\newcommand{\htcop}{H^{13}CO^{+}}$
$\newcommand{\halpha}{H40_{\alpha}}$
$\newcommand{\chthocho}{CH_3OCHO}$
$\newcommand{\chthcho}{CH_3CHO}$
$\newcommand{\chthoh}{CH_3OH}$
$\newcommand{\chii}{H/UC-H\textsc{ii}}$
$\newcommand{\uchii}{UC-H\textsc{ii}}$
$\newcommand{\hchii}{HC-H\textsc{ii}}$
$\newcommand{\hii}{H\textsc{ii}}$
$\newcommand{\CHMC}{s-cHMC}$
$\newcommand{\PCHMC}{w-cHMC}$
$\newcommand{\mdotyr}{M_{\odot}~yr^{-1}}$
$\newcommand{\tred}{\textcolor{red}}$
$\newcommand{\tblue}{\textcolor{blue}}$
$\newcommand{\torange}{\textcolor{orange}}$
$\newcommand{\orcidauthorHL}{0000-0003-3343-9645}$
$\newcommand{\mgt}{\color{magenta}}$
$\newcommand\aj{{\rm{AJ}}}$
$\newcommand\araa{{\rm{ARA\&A}}}$
$\newcommand\apj{{\rm{ApJ}}}$
$\newcommand\icarus{{\rm{Icarus}}}$
$\newcommand\apjs{{\rm{ApJS}}}$
$\newcommand\apjl{{\rm{ApJL}}}$
$\newcommand\apss{{\rm{Ap\&SS}}}$
$\newcommand\aap{{\rm{A\&A}}}$
$\newcommand\aapr{{\rm{A\&AR}}}$
$\newcommand\aaps{{\rm{A\&AS}}}$
$\newcommand\baas{{\rm{BAAS}}}$
$\newcommand\memras{{\rm{MmRAS}}}$
$\newcommand\mnras{{\rm{MNRAS}}}$
$\newcommand\pasp{{\rm{PASP}}}$
$\newcommand\prl{{\rm{Phys. Rev. Lett.}}}$
$\newcommand\jqsrt{{\rm{Journal of Quantitative Spectroscopy and Radiative$
$Transfer}}}$
$\newcommand\actaa{{\rm{Acta Astronomica}}}$</div>



<div id="title">

# $\bf$ The ALMA-QUARKS survey: Evidence of a candidate high-mass prestellar core aside a bright-rimmed cloud IRAS 18290-0924

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.15079-b31b1b.svg)](https://arxiv.org/abs/2511.15079)<mark>Appeared on: 2025-11-20</mark> -  _8 pages, 3 figures, accepted for publication in ApJ_

</div>
<div id="authors">

D. Yang, et al. -- incl., <mark>F. Xu</mark>

</div>
<div id="abstract">

**Abstract:** Although frequently reported in observations, the definitive confirmation of high-mass prestellar cores has remained elusive, presenting a persistent challenge in star formation studies. Using two-band observational data from the 3mm ATOMS and 1.3mm QUARKS surveys, we report a high-mass prestellar core candidate, C2, located on the side of the bright-rimmed cloud IRAS 18290-0924. The C2 core identified from the 3mm continuum data of the ATOMS survey ( $\sim$ 2 $\arcsec$ , $\rm\sim 10000 au$ at 5.3 kpc) has a mass ranging from 27-68 $\msun$ for temperatures 10-22 K within a radius of $\sim$ 2800 au. The highest-resolution ( $\sim$ 0.3 $\arcsec$ , $\rm\sim 1500 au$ ) observations of this source presented to date from the QUARKS survey reveal no evidence of further fragmentation.Further analysis of a total $\sim$ 10 GHz band width of molecular line survey does not find star-formation activity (e.g., outflows, ionized gas) associated with the core, with a few molecular lines of cold gas detected only. Additionally, virial analysis indicates the C2 core is gravitationally bound ( $\alpha_{\rm vir} \sim0.1-0.3$ ) and thus could be undergoing collapse toward star formation.These results strongly establish a candidate for a high-mass prestellar core, contributing to the very limited number of such sources known to date.

</div>

<div id="div_fig1">

<img src="tmp_2511.15079/./spectrum.png" alt="Fig3" width="100%"/>

**Figure 3. -** Average spectra of the C2 core/condensation. (a): average spectra extracted from two ATOMS wideband SPWs over the C2 core. (b): Same as panel a but for
    four QUARKS SPWs over the C2 condensation. (c): zoom-in view of the molecular lines detected in the six SPWs (corresponding to panels a and b). The dashed line indicates the systemic velocity of the C2 core ($\rm$\s$im 84.5 km  s^{-1}$). Note that the absorption dips present in both $^{13}$CO and CO spectra are artifacts arising from the missing flux by ALMA, which will be addressed for future more in-depth studies by combining our QUARKS data with new single-dish observations.
     (*fig:spectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2511.15079/./I18290-0924_1.3mm.png" alt="Fig2" width="100%"/>

**Figure 2. -** QUARKS 1.3 mm dust continuum image (color scale) overlaid with ATOMS 3 mm dust continuum emission (in gray dashed contours, identical to those in Figure \ref{fig:RGB}).
    The black contours are at levels of [3, 6, 12, 24, 48]$\times $\s$igma_{\rm 1.3 mm}$, with $\sigma_{\rm 1.3 mm}$\s$im0.14\rm mJy beam^{-1}$. The blue ellipses correspond to the FWHM sizes of the C2 core estimated from the ATOMS 3 mm image using the task _ CASA imfit_. The synthesized beam sizes of the QUARKS and ATOMS survey are shown in the left and right bottom (panel a), respectively. The 0.01 pc scale bar is shown in the upper right corner. Panel (a): CO (2--1) outflow is overlaid on the QUARKS 1.3 mm dust continuum map. The blue and red contour levels are [3, 6, 12, 24, 48]$\times $\s$igma_{\rm co}$, with $ $\s$igma_{\rm co} $\s$im\rm 0.08 Jy beam^{-1} km s^{-1}$ for the blue lobes and $\sim\rm 0.06 Jy beam^{-1} km s^{-1}$ for the red lobes. The corresponding velocity ranges are plus/minus $\rm  5-30 km s^{-1}$ relative to $\rm V_{LSR}$($\sim\rm 84.5 km s^{-1}$) of the C2 core. Panel (b): zoom-in view of the C2 core. The black ellipses correspond to the FWHM sizes of the condensation estimated from the QUARKS 1.3 mm image using the task _ casa imfit_. (*fig:cont*)

</div>
<div id="div_fig3">

<img src="tmp_2511.15079/./I18290-0924_RGB.png" alt="Fig1" width="100%"/>

**Figure 1. -** Three-colour image of Spitzer 8 $\um$(red), 4.5 $\um$(green), and 3.6 $\um$(blue) of the region associated with I18290.
    The black contours represent the 1.28 GHz MeerKAT emission. The white and green dashed circles show the FoV of the ATOMS and QUARKS survey, respectively. ATOMS 3 mm dust continuum emission is shown as the cyan contours, with levels starting at 3 $\sigma_{\rm 3 mm}$($\rm$\s$im 0.1mJy beam^{-1}$) and increasing as $[6, 12, 24, 48]\times $\s$igma_{\rm 3 mm}$.
    The purple cross symbol shows the location of Core 2  ([Zhang, et. al 2023](https://ui.adsabs.harvard.edu/abs/2023MNRAS.520..322Z)) .
    The synthesized beam size of the ATOMS survey and the 0.5 pc scale bar are shown in the lower left corner and upper right corner, respectively.  (*fig:RGB*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.15079"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# The JWST weather report from the nearest brown dwarfs III: Heterogeneous clouds and Thermochemical instabilities as possible drivers of WISE 1049AB’s spectroscopic variability

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.15667-b31b1b.svg)](https://arxiv.org/abs/2511.15667)<mark>Appeared on: 2025-11-20</mark> -  _Accepted for publication in the Astrophysical Journal (ApJ). 25 pages, 11 figures, 3 tables_

</div>
<div id="authors">

N. Oliveros-Gomez, et al. -- incl., <mark>B. C. Estrada</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We present a new analysis of the spectroscopic variability of WISE J104915.57 $-$ 531906.1AB (WISE 1049AB, L7.5+T0.5), observed using the NIRSpec instrument onboard the James Webb Space Telescope (GO 2965 - PI: Biller).We explored the variability of the dominant molecular bands present in their 0.6--5.3 $\mu$ m spectra ($H_2$ O, $CH_4$ , CO), finding that the B component exhibits a higher maximum deviation than the A component in all the wavelength ranges tested. The light curves reveal wavelength-(atmospheric depth) and possibly chemistry-dependent variability. In particular, for the A component, the variability in the light curves at the wavelengths traced by the $CH_4$ and CO molecular absorption features is higher than that of $H_2$ O, even when both trace similar pressure levels. We concluded that clouds alone are unlikely to explain the increased variability of CO and $CH_4$ with respect to $H_2$ O, suggesting that an additional physical mechanism is needed to explain the observed variability. This mechanism is probably due to thermochemical instabilities.Finally, we provide a visual representation of the 3D atmospheric map reconstructed for both components using the molecular band contributions at different pressure levels and the fit of planetary-scale waves.

</div>

<div id="div_fig1">

<img src="tmp_2511.15667/./Spectrum_AB.png" alt="Fig6" width="100%"/>

**Figure 6. -** _Upper panel:_ Median of the 21 maximum spectra (purple line) and median of the 21 minimum spectra (black line) of WISE 1049A in left panel and WISE 1049B in right panel, taken from "white light curve" (0.6 -- 5.3 $\mu$m) 7 hr NIRSpec/PRISM data. Color segments represent the range of the molecular bands selected in the spectra ($H_2$O-blue, $CH_4$-orange, CO-pink). _Lower panel:_ Max/min flux ratio of the two spectra shown in the upper panel. The horizontal dotted line represents the wavelengths where there is no variability. (*fig:spectrum_A*)

</div>
<div id="div_fig2">

<img src="tmp_2511.15667/./lcfit_paper_A.png" alt="Fig8" width="100%"/>

**Figure 8. -** _Left panel:_ WISE 1049A light curves for different molecular bands of $H_2$O - blue, $CH_4$ - orange, CO - pink, each with the best fit (black), next to each is the number of sinusoidal functions found as seen in the best fit. _Right panel:_ the residual fit (data minus fit). (*fig:lc-A*)

</div>
<div id="div_fig3">

<img src="tmp_2511.15667/./Pressure_structures_A.png" alt="Fig4" width="100%"/>

**Figure 4. -** Artistic representation of the atmospheric layers of WISE 1049A, traced by each molecular feature. Each degraded band represents one planetary wave in each layer, where contrasting the lighter and darker regions with the rotation would generate the variability. (*fig:pressure-structures_A*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.15667"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

131  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
